In [1]:
%pip install -r 002-requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
!docker-compose -f docker-images/docker-compose-milvus.yml up -d
!docker run --rm --name postgres0 -d  -p 5438:5432 -e POSTGRES_HOST_AUTH_METHOD=trust postgres

[+] Running 3/0
 ✔ Container milvus-minio       Running                                    0.0s 
 ✔ Container milvus-etcd        Running                                    0.0s 
 ✔ Container milvus-standalone  Running                                    0.0s 
docker: Error response from daemon: Conflict. The container name "/postgres0" is already in use by container "f1af2dd09abf4844bf22ebfa59280b127c3cc7729e8b5b53b6937029714e9464". You have to remove (or rename) that container to be able to reuse that name.
See 'docker run --help'.


In [3]:
!docker logs postgres0 --tail 6
#!sleep 30

2023-06-05 02:55:28.321 UTC [1] LOG:  listening on IPv6 address "::", port 5432
2023-06-05 02:55:28.322 UTC [1] LOG:  listening on Unix socket "/var/run/postgresql/.s.PGSQL.5432"
2023-06-05 02:55:28.325 UTC [63] LOG:  database system was shut down at 2023-06-05 02:55:28 UTC
2023-06-05 02:55:28.329 UTC [1] LOG:  database system is ready to accept connections
2023-06-05 03:00:28.381 UTC [61] LOG:  checkpoint starting: time
2023-06-05 03:00:35.451 UTC [61] LOG:  checkpoint complete: wrote 72 buffers (0.4%); 0 WAL file(s) added, 0 removed, 0 recycled; write=7.057 s, sync=0.007 s, total=7.072 s; sync files=34, longest=0.001 s, average=0.001 s; distance=374 kB, estimate=374 kB


In [4]:
#Connections to Milvus, BERT and Postgresql
from pymilvus import connections
import psycopg2
connections.connect(host='localhost', port='19530')
conn = psycopg2.connect(host='localhost', port='5438', user='postgres', password='postgres')
cursor = conn.cursor()

In [5]:
TABLE_NAME = "text_collection"
FIELD_NAME = "example_field"

from pymilvus import Collection, CollectionSchema, FieldSchema, DataType

pk = FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True)

field = FieldSchema(name=FIELD_NAME, dtype=DataType.FLOAT_VECTOR, dim=768)
schema = CollectionSchema(fields=[pk,field], description="example collection")

collection = Collection(name=TABLE_NAME, schema=schema)

In [6]:
index_param = {
        "metric_type":"L2",
        "index_type":"IVF_SQ8",
        "params":{"nlist":1024}
    }

collection.create_index(field_name=FIELD_NAME, index_params=index_param)

Status(code=0, message=)

In [7]:
#Deleting previouslny stored table for clean run
drop_table = f"DROP TABLE IF EXISTS {TABLE_NAME}"
cursor.execute(drop_table)
conn.commit()

try:
    sql = f"CREATE TABLE if not exists {TABLE_NAME} (ids bigint, title text, text text);"
    cursor.execute(sql)
    conn.commit()
    print("create postgres table successfully!")
except Exception as e:
    print("can't create a postgres table: ", e)

create postgres table successfully!


In [8]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.preprocessing import normalize

#model = SentenceTransformer('paraphrase-mpnet-base-v2')
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Get article data
data = pd.read_json('news-data/News_Category_Dataset_v3_Medium.json', lines=True)
data = data.replace('\n',' ', regex=True)
data = data.replace('\\|', ' ', regex=True)
                                  
title_data = data['headline'].str.strip().tolist()
text_data = data['short_description'].str.strip().tolist()

sentence_embeddings = model.encode(title_data, batch_size=32)
sentence_embeddings = normalize(sentence_embeddings)
print(type(sentence_embeddings))

Batches:   0%|          | 0/782 [00:00<?, ?it/s]

<class 'numpy.ndarray'>


In [10]:
import numpy as np

em = list(sentence_embeddings)
emb_array = np.array(em)
emb_splits = np.array_split(emb_array, 10)

ids = list()

for emb_split in emb_splits:
    mr_tmp = collection.insert([emb_split.tolist()])
    ids.extend(mr_tmp.primary_keys)
    
#em =list(sentence_embeddings)
#mr = collection.insert([em])
#ids = mr.primary_keys
#dicts ={}

In [11]:
import os

def record_temp_csv(fname, ids, title, text):
    with open(fname,'w') as f:
        for i in range(len(ids)):
            line = str(ids[i]) + "|" + title[i] + "|" + text[i] + "\n"
            f.write(line)

def copy_data_to_pg(table_name, fname, conn, cur):
    fname = os.path.join(os.getcwd(),fname)
    try:
        sql = f"COPY {table_name} FROM STDIN (QUOTE E'\u0007', FORMAT 'csv', DELIMITER '|')"
        cursor.copy_expert(sql, open(fname, "r"))
        conn.commit()
        print("Inserted into Postgress Sucessfully!")
    except Exception as e:
        print("Copy Data into Postgress failed: ", e)
        
DATA_WITH_IDS = 'news-data/temp-to-load.csv'   

record_temp_csv(DATA_WITH_IDS, ids, title_data, text_data)
copy_data_to_pg(TABLE_NAME, DATA_WITH_IDS, conn, cursor)

Inserted into Postgress Sucessfully!


In [12]:
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}

query_vec = []

title = 'War in Ukraine'

query_embeddings = []
embed = model.encode(title)
embed = embed.reshape(1,-1)
embed = normalize(embed)
query_embeddings = embed.tolist()

collection.load()
results = collection.search(query_embeddings, FIELD_NAME, param=search_params, limit=20, expr=None)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [13]:
similar_titles = []

#if results[0][0].distance > 0.7:
#    print("There are no similar questions in the database, here are the closest matches:")
#else:
#    print("There are similar questions in the database, here are the closest matches: ")
    
print("Here are the closest article matches: ")

for result in results[0]:
    sql = f"select title from {TABLE_NAME} where ids = " + str(result.id) + ";"
    print(result.id)
    cursor.execute(sql)
    rows=cursor.fetchall()
    if len(rows):
        if rows[0][0]:
            similar_titles.append((rows[0][0], result.distance))
            print((rows[0][0], result.distance))



Here are the closest article matches: 
441957411561412503
('Ukraine Documents Alleged Atrocities By Retreating Russians', 0.47776588797569275)
441957411561412462
('Russia Appoints New Ukraine War Commander With Record Of Brutality', 0.52095627784729)
441957411561412708
("Europe Braces For Refugees As Russia Attacks Ukraine's Capital", 0.5226385593414307)
441957411561412660
('Mariupol Diaries: Scenes Of Despair And Resolve In The Besieged Ukrainian City', 0.5455607175827026)
441957411561412578
('Russian Military Slog In Ukraine A ‘Dreadful Mess’ For Putin', 0.5525690317153931)
441957411561412715
('Cyberattacks Accompany Russian Military Assault On Ukraine', 0.575350284576416)
441957411561412735
('Russia Extends Troop Drills Near Ukraine As Violence Spikes', 0.6013284921646118)
441957411561412711
('Ukraine Attack Leaves Baltics Wondering: Are We Next?', 0.6058027744293213)
441957411561426694
("What's Driving The Recent Carnage In Kabul", 0.6065389513969421)
441957411561412353
('Ukraine F

### Cleanup of docker images

In [ ]:
%%bash
L_END_DOCKER=1

if [[ ${L_END_DOCKER} -eq 1 ]]; then
    docker-compose -f docker-images/docker-compose-milvus.yml down --remove-orphans
    docker stop postgres0
fi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


 Container milvus-standalone  Stopping


### Articles cited (For data purposes)

1. Misra, Rishabh. "News Category Dataset." arXiv preprint arXiv:2209.11429 (2022).
2. Misra, Rishabh and Jigyasa Grover. "Sculpting Data for ML: The first act of Machine Learning." ISBN 9798585463570 (2021).